In [10]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
import jsonlines

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Input, Bidirectional, Conv1D, MaxPool1D
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

In [11]:
result = pd.read_pickle('Dataset/Cleanded.pkl')

In [12]:
tv        = result['TV'].tolist()
fintech   = result['FinTech'].tolist()
iot       = result['IoT'].tolist()
insurance = result['Insurance'].tolist()
mbb       = result['MBB'].tolist()
fbb       = result['FBB'].tolist()
e_sport   = result['E-Sport'].tolist()
other     = result['Other'].tolist()

In [13]:
import fasttext

# load saved into python
model = fasttext.load_model('cc.th.300.bin')

In [14]:
def text_reshaper(text_list, reshaped_text_list, MAX_LENGTH):
  for i in range(text_list.shape[0]-1):
     if len(text_list[i]) >= MAX_LENGTH:
         reshaped_text_list[i, :] = text_list[i][0:MAX_LENGTH]
     else:
         reshaped_text_list[i, 0:len(text_list[i])] = text_list[i]
         reshaped_text_list[i, len(text_list[i]):] = " "
         
  return reshaped_text_list

In [15]:
lines_of_file = []
lines = []
with open('Dataset/tokenize.txt', 'r', encoding="utf-8") as f:
  lines_of_file = f.readlines()
  for oneline in lines_of_file:
    lines.append(oneline.split(" "))

In [16]:
# Clear the space element out of token list
for item in lines:
  for element in item:
    if(element == ''):
      item.remove(element)

In [17]:
# RESHAPE THE DATA 
MAX_LENGTH = 300

y_train = np.array(mbb[:1362])
y_test = np.array(mbb[1362: 19450])

texts_token_list_reshaped = np.zeros([1945, MAX_LENGTH], dtype=np.object)
texts_token_list = np.array(lines, dtype=np.object)
texts_token_list_reshaped = text_reshaper(texts_token_list, texts_token_list_reshaped, MAX_LENGTH)

In [18]:
input = Input((MAX_LENGTH,model.get_dimension()))

x = Conv1D(32,3,padding='same',activation='relu')(input)
x = MaxPool1D()(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(64, return_sequences=False))(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)

out = Dense(2, activation='softmax')(x)

In [19]:
wn_sentiment_model = Model(input, out)
wn_sentiment_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 32)           28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 150, 32)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 150, 256)          164864    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)               

In [20]:
# Create embeded input vector (takes a while)
fasttext_vec = np.zeros((1945, MAX_LENGTH, 300), dtype=np.float_)

for text in range(fasttext_vec.shape[0]):
  for word in range(fasttext_vec.shape[1]):
      fasttext_vec[text][word] = model.get_word_vector(texts_token_list_reshaped[text][word])


TypeError: getWordVector(): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.fasttext, arg0: fasttext_pybind.Vector, arg1: str) -> None

Invoked with: <fasttext_pybind.fasttext object at 0x0000019B408B98B0>, <fasttext_pybind.Vector object at 0x0000019D0C731F30>, 0

In [21]:
x_train = fasttext_vec[:1362]
x_test = fasttext_vec[1362: 1945]

y_train_tv = tv[:1362]
y_test_tv = tv[1362: 1945]
y_train_fintech = fintech[:1362]
y_test_fintech = fintech[1362: 1945]
y_train_iot = iot[:1362]
y_test_iot = iot[1362: 1945]
y_train_insurance = insurance[:1362]
y_test_insurance = insurance[1362: 1945]
y_train_mbb = mbb[:1362]
y_test_mbb = mbb[1362: 1945]
y_train_fbb = fbb[:1362]
y_test_fbb = fbb[1362: 1945]
y_train_e_sport = e_sport[:1362]
y_test_e_sport = e_sport[1362: 1945]
y_train_other = other[:1362]
y_test_other = other[1362: 1945]

In [22]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train_tv = np.array(y_train_tv)
y_test_tv = np.array(y_test_tv)
y_train_fintech = np.array(y_train_fintech)
y_test_fintech = np.array(y_test_fintech)
y_train_iot = np.array(y_train_iot)
y_test_iot = np.array(y_test_iot)
y_train_insurance = np.array(y_train_insurance)
y_test_insurance = np.array(y_test_insurance)
y_train_mbb = np.array(y_train_mbb)
y_test_mbb = np.array(y_test_mbb)
y_train_fbb = np.array(y_train_fbb)
y_test_fbb = np.array(y_test_fbb)
y_train_e_sport = np.array(y_train_e_sport)
y_test_e_sport = np.array(y_test_e_sport)
y_train_other = np.array(y_train_other)
y_test_other = np.array(y_test_other)

In [19]:
wn_sentiment_model.compile(
    optimizer=RMSprop(learning_rate=0.0005, momentum=0.9),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [20]:
model_checkpoint_callback_tv = tf.keras.callbacks.ModelCheckpoint(
    filepath="./checkpoint_tv",
    save_best_only=True)

In [21]:
model_tv = wn_sentiment_model.fit(x_train, y_train_tv, validation_data=(x_test,y_test_tv), epochs=10, callbacks=model_checkpoint_callback_tv)

Epoch 1/10
43/43 [==============================] - ETA: 0s - loss: 0.3437 - accuracy: 0.9148WARNING:tensorflow:From C:\Users\James\Anaconda3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: .\checkpoint_tv\assets
43/43 [==============================] - 21s 493ms/step - loss: 0.3437 - accuracy: 0.9148 - val_loss: 0.2983 - val_accuracy: 0.9262
Epoch 2/10
43/43 [==============================] - 19s 451ms/step - loss: 0.3076 - accuracy: 0.9148 - val_loss: 0.2773 - val_accuracy: 0.9262
Epoch 3/10
43/43 [==============================] - 20s 456ms/step - loss: 0.3009 - accuracy: 0.91

In [22]:
model_tv = keras.models.load_model("./checkpoint_tv")

In [23]:
model_tv.evaluate(x_test,  y_test_tv, verbose=2)

19/19 - 0s - loss: 0.2038 - accuracy: 0.9520


[0.20377884805202484, 0.9519725441932678]

In [24]:
# Fintech

model_checkpoint_callback_fintech = tf.keras.callbacks.ModelCheckpoint(
    filepath="./checkpoint_fintech",
    save_best_only=True)

In [25]:
wn_sentiment_model.fit(x_train, 
                      y_train_fintech, 
                      validation_data=(x_test,y_test_fintech), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_fintech)

Epoch 1/10
43/43 [==============================] - 19s 436ms/step - loss: 0.3504 - accuracy: 0.8928 - val_loss: 0.3046 - val_accuracy: 0.9194
Epoch 2/10
43/43 [==============================] - 20s 457ms/step - loss: 0.2974 - accuracy: 0.9053 - val_loss: 0.2752 - val_accuracy: 0.9194
Epoch 3/10
43/43 [==============================] - 3s 63ms/step - loss: 0.2664 - accuracy: 0.9053 - val_loss: 0.3238 - val_accuracy: 0.9194
Epoch 4/10
43/43 [==============================] - 3s 59ms/step - loss: 0.2493 - accuracy: 0.9053 - val_loss: 0.2950 - val_accuracy: 0.9194
Epoch 5/10
43/43 [==============================] - 3s 59ms/step - loss: 0.2358 - accuracy: 0.9053 - val_loss: 0.3128 - val_accuracy: 0.9194
Epoch 6/10
43/43 [==============================] - 3s 64ms/step - loss: 0.2191 - accuracy: 0.9053 - val_loss: 0.3400 - val_accuracy: 0.9194
Epoch 7/10
43/43 [==============================] - 3s 59ms/step - loss: 0.2039 - accuracy: 0.9068 - val_loss: 0.3389 - val_accuracy: 0.9039
Epoch 8/1

In [26]:
model_fintech = keras.models.load_model("./checkpoint_fintech")

model_fintech.evaluate(x_test,  y_test_fintech, verbose=2)

19/19 - 0s - loss: 0.2752 - accuracy: 1.0000


[0.2752346992492676, 1.0]

In [27]:
# IoT

model_checkpoint_callback_iot = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_iot",
                                                save_best_only=True)

In [28]:
# IoT

wn_sentiment_model.fit(x_train, 
                      y_train_iot, 
                      validation_data=(x_test,y_test_iot), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_iot)

Epoch 1/10
43/43 [==============================] - 21s 487ms/step - loss: 0.5057 - accuracy: 0.8179 - val_loss: 0.4599 - val_accuracy: 0.8233
Epoch 2/10
43/43 [==============================] - 21s 487ms/step - loss: 0.4398 - accuracy: 0.8289 - val_loss: 0.4542 - val_accuracy: 0.8233
Epoch 3/10
43/43 [==============================] - 21s 491ms/step - loss: 0.4032 - accuracy: 0.8289 - val_loss: 0.4475 - val_accuracy: 0.8233
Epoch 4/10
43/43 [==============================] - 3s 61ms/step - loss: 0.3554 - accuracy: 0.8414 - val_loss: 0.4609 - val_accuracy: 0.8130
Epoch 5/10
43/43 [==============================] - 3s 61ms/step - loss: 0.3315 - accuracy: 0.8502 - val_loss: 0.4890 - val_accuracy: 0.7753
Epoch 6/10
43/43 [==============================] - 3s 59ms/step - loss: 0.3250 - accuracy: 0.8517 - val_loss: 0.5231 - val_accuracy: 0.8010
Epoch 7/10
43/43 [==============================] - 3s 63ms/step - loss: 0.3022 - accuracy: 0.8642 - val_loss: 0.6060 - val_accuracy: 0.7942
Epoch 8

In [29]:
# IoT

model_iot = keras.models.load_model("./checkpoint_iot")

model_iot.evaluate(x_test,  y_test_iot, verbose=2)

19/19 - 1s - loss: 0.4475 - accuracy: 1.0000


[0.4474872648715973, 1.0]

In [30]:
# Insurance

model_checkpoint_callback_insurance = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_insurance",
                                                save_best_only=True)

In [31]:
# Insurance

wn_sentiment_model.fit(x_train, 
                      y_train_insurance, 
                      validation_data=(x_test,y_test_insurance), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_insurance)

Epoch 1/10
43/43 [==============================] - 21s 486ms/step - loss: 0.2086 - accuracy: 0.9435 - val_loss: 0.1660 - val_accuracy: 0.9640
Epoch 2/10
43/43 [==============================] - 20s 470ms/step - loss: 0.1477 - accuracy: 0.9684 - val_loss: 0.1655 - val_accuracy: 0.9640
Epoch 3/10
43/43 [==============================] - 2s 57ms/step - loss: 0.1336 - accuracy: 0.9684 - val_loss: 0.1969 - val_accuracy: 0.9640
Epoch 4/10
43/43 [==============================] - 2s 57ms/step - loss: 0.1375 - accuracy: 0.9684 - val_loss: 0.2002 - val_accuracy: 0.9640
Epoch 5/10
43/43 [==============================] - 2s 57ms/step - loss: 0.1172 - accuracy: 0.9684 - val_loss: 0.2602 - val_accuracy: 0.9640
Epoch 6/10
43/43 [==============================] - 2s 58ms/step - loss: 0.1071 - accuracy: 0.9684 - val_loss: 0.2992 - val_accuracy: 0.9640
Epoch 7/10
43/43 [==============================] - 3s 58ms/step - loss: 0.1117 - accuracy: 0.9684 - val_loss: 0.3930 - val_accuracy: 0.9640
Epoch 8/1

In [32]:
# Insurance

model_insurance = keras.models.load_model("./checkpoint_insurance")

model_insurance.evaluate(x_test,  y_test_insurance, verbose=2)

19/19 - 0s - loss: 0.1655 - accuracy: 1.0000


[0.1655135154724121, 1.0]

In [33]:
# MBB

model_checkpoint_callback_mbb = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_mbb",
                                                save_best_only=True)

In [34]:
# MBB

wn_sentiment_model.fit(x_train, 
                      y_train_mbb, 
                      validation_data=(x_test,y_test_mbb), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_mbb)

Epoch 1/10
43/43 [==============================] - 21s 497ms/step - loss: 1.2457 - accuracy: 0.5374 - val_loss: 0.6825 - val_accuracy: 0.5660
Epoch 2/10
43/43 [==============================] - 21s 495ms/step - loss: 0.6786 - accuracy: 0.5683 - val_loss: 0.6725 - val_accuracy: 0.5660
Epoch 3/10
43/43 [==============================] - 21s 491ms/step - loss: 0.6725 - accuracy: 0.5690 - val_loss: 0.6634 - val_accuracy: 0.5660
Epoch 4/10
43/43 [==============================] - 21s 493ms/step - loss: 0.6492 - accuracy: 0.5896 - val_loss: 0.6454 - val_accuracy: 0.5918
Epoch 5/10
43/43 [==============================] - 22s 511ms/step - loss: 0.6123 - accuracy: 0.6586 - val_loss: 0.6255 - val_accuracy: 0.6381
Epoch 6/10
43/43 [==============================] - 21s 491ms/step - loss: 0.5812 - accuracy: 0.6938 - val_loss: 0.6004 - val_accuracy: 0.6432
Epoch 7/10
43/43 [==============================] - 3s 63ms/step - loss: 0.5442 - accuracy: 0.7217 - val_loss: 0.6097 - val_accuracy: 0.6690
E

In [35]:
# MBB

model_mbb = keras.models.load_model("./checkpoint_mbb")

model_mbb.evaluate(x_test,  y_test_mbb, verbose=2)

19/19 - 0s - loss: 0.5695 - accuracy: 0.5540


[0.5694996118545532, 0.5540308952331543]

In [36]:
# FBB

model_checkpoint_callback_fbb = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_fbb",
                                                save_best_only=True)

In [37]:
# FBB

wn_sentiment_model.fit(x_train, 
                      y_train_fbb, 
                      validation_data=(x_test,y_test_fbb), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_fbb)

Epoch 1/10
43/43 [==============================] - 21s 495ms/step - loss: 0.5748 - accuracy: 0.7981 - val_loss: 0.4501 - val_accuracy: 0.8302
Epoch 2/10
43/43 [==============================] - 3s 65ms/step - loss: 0.4541 - accuracy: 0.8399 - val_loss: 0.4547 - val_accuracy: 0.8302
Epoch 3/10
43/43 [==============================] - 21s 487ms/step - loss: 0.4394 - accuracy: 0.8399 - val_loss: 0.4481 - val_accuracy: 0.8302
Epoch 4/10
43/43 [==============================] - 21s 491ms/step - loss: 0.4276 - accuracy: 0.8399 - val_loss: 0.4468 - val_accuracy: 0.8302
Epoch 5/10
43/43 [==============================] - 3s 61ms/step - loss: 0.4067 - accuracy: 0.8414 - val_loss: 0.4607 - val_accuracy: 0.8285
Epoch 6/10
43/43 [==============================] - 3s 65ms/step - loss: 0.3805 - accuracy: 0.8436 - val_loss: 0.4667 - val_accuracy: 0.8113
Epoch 7/10
43/43 [==============================] - 3s 62ms/step - loss: 0.3630 - accuracy: 0.8421 - val_loss: 0.4854 - val_accuracy: 0.7804
Epoch 8

In [38]:
# FBB

model_fbb = keras.models.load_model("./checkpoint_fbb")

model_fbb.evaluate(x_test,  y_test_fbb, verbose=2)

19/19 - 0s - loss: 0.4468 - accuracy: 1.0000


[0.4467894732952118, 1.0]

In [39]:
# E-sport

model_checkpoint_callback_e_sport = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_e_sport",
                                                save_best_only=True)

In [40]:
# E-sport

wn_sentiment_model.fit(x_train, 
                      y_train_e_sport, 
                      validation_data=(x_test,y_test_e_sport), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_e_sport)

Epoch 1/10
43/43 [==============================] - 22s 502ms/step - loss: 0.1410 - accuracy: 0.9648 - val_loss: 0.0947 - val_accuracy: 0.9846
Epoch 2/10
43/43 [==============================] - 22s 504ms/step - loss: 0.1007 - accuracy: 0.9772 - val_loss: 0.0685 - val_accuracy: 0.9846
Epoch 3/10
43/43 [==============================] - 3s 65ms/step - loss: 0.0865 - accuracy: 0.9772 - val_loss: 0.0790 - val_accuracy: 0.9846
Epoch 4/10
43/43 [==============================] - 3s 62ms/step - loss: 0.0558 - accuracy: 0.9772 - val_loss: 0.0813 - val_accuracy: 0.9846
Epoch 5/10
43/43 [==============================] - 3s 69ms/step - loss: 0.0392 - accuracy: 0.9765 - val_loss: 0.0978 - val_accuracy: 0.9863
Epoch 6/10
43/43 [==============================] - 3s 69ms/step - loss: 0.0320 - accuracy: 0.9728 - val_loss: 0.1637 - val_accuracy: 0.9811
Epoch 7/10
43/43 [==============================] - 3s 67ms/step - loss: 0.0482 - accuracy: 0.9816 - val_loss: 0.1213 - val_accuracy: 0.9708
Epoch 8/1

In [41]:
# E-sport

model_e_sport = keras.models.load_model("./checkpoint_e_sport")

model_e_sport.evaluate(x_test,  y_test_e_sport, verbose=2)

19/19 - 0s - loss: 0.0685 - accuracy: 1.0000


[0.06852228939533234, 1.0]

In [42]:
# Other

model_checkpoint_callback_other = tf.keras.callbacks.ModelCheckpoint(
                                                filepath="./checkpoint_other",
                                                save_best_only=True)

In [43]:
# Other

wn_sentiment_model.fit(x_train, 
                      y_train_other, 
                      validation_data=(x_test,y_test_other), 
                      epochs=10, 
                      callbacks=model_checkpoint_callback_other)

Epoch 1/10
43/43 [==============================] - 22s 505ms/step - loss: 1.1628 - accuracy: 0.6608 - val_loss: 0.6183 - val_accuracy: 0.6998
Epoch 2/10
43/43 [==============================] - 21s 488ms/step - loss: 0.6205 - accuracy: 0.6982 - val_loss: 0.6175 - val_accuracy: 0.6998
Epoch 3/10
43/43 [==============================] - 23s 535ms/step - loss: 0.6130 - accuracy: 0.6982 - val_loss: 0.6101 - val_accuracy: 0.6998
Epoch 4/10
43/43 [==============================] - 3s 71ms/step - loss: 0.6032 - accuracy: 0.6982 - val_loss: 0.6109 - val_accuracy: 0.6998
Epoch 5/10
43/43 [==============================] - 23s 533ms/step - loss: 0.5706 - accuracy: 0.6982 - val_loss: 0.6040 - val_accuracy: 0.6998
Epoch 6/10
43/43 [==============================] - 25s 570ms/step - loss: 0.5451 - accuracy: 0.6982 - val_loss: 0.6009 - val_accuracy: 0.6998
Epoch 7/10
43/43 [==============================] - 3s 66ms/step - loss: 0.4994 - accuracy: 0.7195 - val_loss: 0.6603 - val_accuracy: 0.7118
Epo

In [44]:
# Other

model_other = keras.models.load_model("./checkpoint_other")

model_other.evaluate(x_test,  y_test_other, verbose=2)

19/19 - 1s - loss: 0.5784 - accuracy: 0.5352


[0.5783796906471252, 0.5351629257202148]

# Model f1-score

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train_tv = np.array(y_train_tv)
y_test_tv = np.array(y_test_tv)
y_train_fintech = np.array(y_train_fintech)
y_test_fintech = np.array(y_test_fintech)
y_train_iot = np.array(y_train_iot)
y_test_iot = np.array(y_test_iot)
y_train_insurance = np.array(y_train_insurance)
y_test_insurance = np.array(y_test_insurance)
y_train_mbb = np.array(y_train_mbb)
y_test_mbb = np.array(y_test_mbb)
y_train_fbb = np.array(y_train_fbb)
y_test_fbb = np.array(y_test_fbb)
y_train_e_sport = np.array(y_train_e_sport)
y_test_e_sport = np.array(y_test_e_sport)
y_train_other = np.array(y_train_other)
y_test_other = np.array(y_test_other)

In [4]:
model_tv = keras.models.load_model("./checkpoint_tv")
model_fintech = keras.models.load_model("./checkpoint_fintech")
model_iot = keras.models.load_model("./checkpoint_iot")
model_insurance = keras.models.load_model("./checkpoint_insurance")
model_mbb = keras.models.load_model("./checkpoint_mbb")
model_fbb = keras.models.load_model("./checkpoint_fbb")
model_e_sport = keras.models.load_model("./checkpoint_e_sport")
model_other = keras.models.load_model("./checkpoint_other")

In [5]:
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

TV model

In [23]:
pred = np.argmax(model_tv.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_tv, pred, average='macro'))
print("precision :",average_precision_score(y_test_tv, pred))

f1-score : 0.6925461444465881
precision : 0.2349045206658195


Fintech model

In [24]:
pred = np.argmax(model_fintech.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_fintech, pred, average='macro'))
print("precision :",average_precision_score(y_test_fintech, pred))

f1-score : 0.47899910634495085
precision : 0.08061749571183534


IoT model

In [25]:
pred = np.argmax(model_iot.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_iot, pred, average='macro'))
print("precision :",average_precision_score(y_test_iot, pred))

f1-score : 0.45155221072436497
precision : 0.17667238421955403


Insurance Model

In [26]:
pred = np.argmax(model_insurance.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_insurance, pred, average='macro'))
print("precision :",average_precision_score(y_test_insurance, pred))

f1-score : 0.4908296943231441
precision : 0.036020583190394515


MBB Model

In [27]:
pred = np.argmax(model_mbb.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_mbb, pred, average='macro'))
print("precision :",average_precision_score(y_test_mbb, pred))

f1-score : 0.6877766007532957
precision : 0.7011584641773321


FBB Model

In [31]:
pred = np.argmax(model_other.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_other, pred, average='macro'))
print("precision :",average_precision_score(y_test_other, pred))

f1-score : 0.6586821126058794
precision : 0.4297931101830008


E-sport Model

In [30]:
pred = np.argmax(model_e_sport.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_e_sport, pred, average='macro'))
print("precision :",average_precision_score(y_test_e_sport, pred))

f1-score : 0.49611063094209157
precision : 0.015437392795883362


Others model

In [82]:
pred = np.argmax(model_other.predict(x_test), axis=1)  # from log probabilities to 0 or 1
print("f1-score :",f1_score(y_test_other, pred, average='macro'))
print("precision :",average_precision_score(y_test_other, pred))

f1-score : 0.6586821126058794
precision : 0.4297931101830008
